In [ ]:
import os,sys,re
import numpy as np
import pandas as pd
import scanpy as sc

In [ ]:
p = 'path'

In [ ]:
sc.settings.verbosity = 3
sc.logging.print_versions()
sc.settings.set_figure_params(dpi=80)

results_file = '%s/scanpy_results.h5ad'%p

adata = sc.read_10x_mtx(
    '%s/'%p,
    var_names='gene_symbols',
    cache=False)    
adata


In [ ]:
sc.pp.filter_cells(adata, min_genes=5)
sc.pp.filter_genes(adata, min_cells=500)
adata.var['mt'] = adata.var_names.str.startswith('MT-')
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

sc.pp.log1p(adata)
adata.raw = adata
sc.pp.regress_out(adata, ['total_counts'])
sc.pp.scale(adata, max_value=15)

sc.tl.pca(adata, svd_solver='arpack')
sc.pl.pca(adata)

In [ ]:
sc.pp.neighbors(adata, n_neighbors=15, n_pcs=17)
sc.tl.umap(adata)

sc.tl.leiden(adata)

sc.pl.umap(adata, color=['leiden']) 

In [ ]:
#plotting expression of individual genes
sc.settings.set_figure_params(dpi=250)
 
sc.pl.umap(adata, color=['cdrA'],add_outline=False,vmin=0,vmax='p99',use_raw=False,color_map = 'Blues') # key_added='clusters',resolution = 1


In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='t-test',max_iter=100000) #identify differentially expressed genes 
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
result = adata.uns['rank_genes_groups']
result.keys()
groups = result['names'].dtype.names
data_to_write = pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names','logfoldchanges','pvals_adj']})

data_to_write.to_csv('%s//npc_15_cluster_exp_wilcoxon.txt'%p,sep = '\t')

In [ ]:
#write to file
adata.write(results_file)
adata.write_csvs(results_file[:-5], )